## LLM Schema Scraper

1. Install all required modules (cohere, langchain) from `requirements.txt` first before running using `pip install -r requirements.txt`

In [1]:
from langchain_cohere import ChatCohere

llm = ChatCohere()

In [2]:
llm.invoke("how can langsmith help with documentation retreival of LLMs?")

AIMessage(content="Langsmith can help with testing in a number of ways: \n\n- **Automated Testing**: Langsmith can be used to automate various testing tasks. With its natural language processing capabilities, Langsmith can understand and generate human language. This means it can be trained to automatically generate test cases, write test scripts, and even simulate user interactions with an interface, reducing the need for manual effort in these areas. \n\n- **Test Case Generation**: Langsmith can assist in creating comprehensive test case scenarios. By inputting the requirements and desired functionality, Langsmith can suggest test cases that cover a wide range of possibilities, including edge cases and boundary conditions, ensuring more robust testing. \n\n- **Test Data Generation**: Langsmith can generate large volumes of realistic and varied test data. This is particularly useful for testing applications that handle user-generated content, as it can simulate diverse user inputs, he

### Documentation Loader

This uses the class `WebBaseLoader` to load a LLM documentation via URL

In [9]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://ai.google.dev/models/gemini") # url if the model documentation

docs = loader.load()

In [10]:
from langchain_cohere.embeddings import CohereEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter

# Index into a vectorstore. This requires an embedding model (Cohere) and a vectorstore

embeddings = CohereEmbeddings()

# Use simple local vectorstore, FAISS, to ingest documents into a vectorstore

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [11]:
# Creating a retrieval chain from vectorstore
# Chain will take a question, look up relevant documents, then pass them with original question into Cohere

from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [12]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

response = retrieval_chain.invoke({"input": "what is the model schema data of Gemini 1.0 Pro"})
print(response["answer"])

The model schema data for the Gemini 1.0 Pro model includes the following:

- **Input**: Text
- **Output**: Text
- **Model size**: Gemini 1.0 Pro is described as balancing capability and efficiency.
- **Model code**: models/gemini-1.0-pro
- **Capabilities**: Generates text and can handle multi-turn conversational format. It can also handle zero, one, and few-shot tasks.
- **Supported generation methods**: generateContent
- **Input token limit**: 30720
- **Output token limit**: 2048
- **Rate limit**: 60 requests per minute
- **Latest stable version**: gemini-1.0-pro

This information is provided in the context you supplied, which appears to be a documentation page for the Gemini API and its various models.
